# Análisis y Mejora de Detección de Links

Este notebook se centra en analizar y mejorar los métodos de detección de links de la clase `DescubrirSitios`, especialmente para filtrar links irrelevantes como categorías de WordPress, posts de autores, etc.

## 1. Cargar Datos de SITIOS_RELEVANTES

In [ ]:
# Instalación de dependencias
# %pip install crawl4ai beautifulsoup4 


In [9]:
# Importaciones necesarias
import sys
import os
import re
import asyncio
import nest_asyncio
nest_asyncio.apply()
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from crawl4ai import AsyncWebCrawler

# Configurar policy para Windows/VSCode
if sys.platform == "win32":
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())

if sys.platform.startswith("win") and sys.version_info >= (3, 8):

            try:
                from asyncio import (
                    WindowsProactorEventLoopPolicy,
                    WindowsSelectorEventLoopPolicy,
                )
            except ImportError:
                pass
                # not affected
            else:
                if type(asyncio.get_event_loop_policy()) is WindowsProactorEventLoopPolicy:
                    # WindowsProactorEventLoopPolicy is not compatible with tornado 6
                    # fallback to the pre-3.8 default of Selector
                    asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())
                    print(f'do not change to Selector Event Loop')


# Agregar el directorio padre al path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))



In [10]:
# Importar desde el módulo
from descubrimiento import DescubrirSitios, PATRONES_INTERES, DEPARTAMENTOS_CONOCIDOS, DOMINIO, URLS_BASE

# 1. Crear instancia de la clase:
descubridor = DescubrirSitios()

## 1. Análisis del Método Original

Primero vamos a examinar el método `extraer_links_relevantes` actual y probarlo con algunos ejemplos.

In [ ]:
# Crear instancia para probar
descubridor = DescubrirSitios()

# Ver los patrones actuales
print("🎯 PATRONES DE INTERÉS ACTUALES:")
for i, patron in enumerate(PATRONES_INTERES, 1):
    print(f"  {i}. {patron}")

print("\n🏛️ DEPARTAMENTOS CONOCIDOS:")
for i, depto in enumerate(DEPARTAMENTOS_CONOCIDOS, 1):
    print(f"  {i}. {depto}")

print("\n🌐 DOMINIOS PERMITIDOS:")
for i, dominio in enumerate(DOMINIO, 1):
    print(f"  {i}. {dominio}")

In [11]:
async def procesar_url_simple(url):
    async with AsyncWebCrawler() as crawler:
        resultado = await descubridor.procesar_url(crawler, url)
        return resultado

In [12]:
url_ejemplo = "https://lcd.exactas.uba.ar/"

links = asyncio.run(procesar_url_simple(url_ejemplo))
print(links)

[INIT].... → Crawl4AI 0.7.2 

🔍 Analizando: https://lcd.exactas.uba.ar/


[FETCH]... ↓ https://lcd.exactas.uba.ar/                                                                          |
✓ | ⏱: 2.61s 

[SCRAPE].. ◆ https://lcd.exactas.uba.ar/                                                                          |
✓ | ⏱: 0.03s 

[COMPLETE] ● https://lcd.exactas.uba.ar/                                                                          |
✓ | ⏱: 2.64s 

{'url': 'https://lcd.exactas.uba.ar/', 'titulo': 'Licenciatura en Datos – Exactas – UBA – Carrera de la Facultad de Ciencias Exactas y Naturales', 'tecnologia': {'cms': 'wordpress', 'framework': 'desconocido', 'complejidad': 'compleja'}, 'contenido_materias': {'tiene_materias': True, 'tiene_horarios': True, 'tiene_correlativas': True, 'tiene_programas': True, 'cantidad_materias_estimada': 15, 'tipos_informacion': ['horarios', 'correlativas', 'programas']}, 'links_encontrados': ['https://lcd.exactas.uba.ar/materias/', 'https://lcd.exactas.uba.ar/materias-obligatorias/', 'https://lcd.exactas.uba.ar/materias-optativas/', 'https://lcd.exactas.uba.ar/horarios-de-consulta/', 'https://lcd.exactas.uba.ar/zoom-de-metodos-para-datos-moleculares-de-alta-dimension-estadistica-computacional-y-prediccion-de-riesgos/', 'https://lcd.exactas.uba.ar/zoom-de-metodos-para-datos-moleculares-de-alta-dimension-estadistica-computacional-y-prediccion-de-riesgos/#respond', 'https://lcd.exactas.uba.ar/materias/#